In [4]:
import pandas as pd
import json
import seaborn as sns
import requests

In [2]:
with open("../new-york-open-data-explorer/src/datasets.json",'r') as f:
    datasets = json.load(f)

In [3]:
len(datasets)

2804

In [4]:
len(set([2,4,5,6,7]).intersection(set([2,5])))

2

In [34]:
def get_dataset_size(dataset):
    url = 'https://data.cityofnewyork.us/resource/{}.json?$select=count(*)'.format(dataset['resource']['id'])
    reply = requests.get(url).json()
    try:
        count  =  int(reply[0]['count'])
        return count
    except:
        print('Issue with ',dataset['resource']['id'])
        print("reply was ",reply)
        return 0


In [24]:
black_list = set(['year','latitude','longitude','id'])

In [25]:
white_list = set(['bin','bbl','school_name','council_district','nta','dbn','census_tract','district','community_council'])

In [26]:
def find_joinable(dataset):
    columns = set( [ a.lower() for a in dataset['resource']['columns_field_name']])
    result = []
    for comp_dataset in datasets:
        if(comp_dataset['resource']['id'] == dataset['resource']['id']):
            continue
        comp_columns = set([a.lower() for a in comp_dataset['resource']['columns_field_name']])
        inter = columns.intersection(comp_columns)
        inter = inter.intersection(white_list)
        no_matches = len(inter)
        if(no_matches>0):
            result.append({'o': dataset['resource']['id'],
                           'd':comp_dataset['resource']['id'],
                           'col':no_matches
                          })
        
    return result

In [27]:
joins = []
for dataset in datasets:
    joins.extend(find_joinable(dataset))

In [28]:
joins = pd.DataFrame(joins)

In [29]:
potential_join_datasets = pd.DataFrame(joins).groupby('o').count()[['d']]

In [30]:
potential_join_datasets

,d
o,
22rr-ujq3,304
23z9-6uk9,737
25aa-q86c,354
27h8-t3wt,126
29bv-qqsy,304
...,...
ywiv-5gyw,354
yz7z-iupz,126
zmut-au2w,384


In [31]:
def get_agency(dataset):
    metadata = dataset['classification']['domain_metadata']
    hits= [entry['value'] for entry in metadata if entry['key'] == 'Dataset-Information_Agency']
    if(len(hits)>0):
        return hits[0]
    return None

In [37]:
counts = [ get_dataset_size(dataset) for dataset in datasets]

Issue with  wyhw-ykq2
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  ixe9-9fcq
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  x4hu-rz9v
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  yeut-b48d
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  q48k-6suv
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  6etc-vpfd
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  fk5e-4bf4
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  c5u8-5c47
reply was  {'error': True, 'message': 'Cannot create columns on non-tabular datasets'}
Issue with  28sw-qd77
reply was  {'code': 'dataset.missing', 'error': True, 'message': 'Not found', 'data': {'id': '28sw-qd77'}}

In [41]:
len(counts)

2804

In [36]:
counts

NameError: name 'counts' is not defined

In [63]:
data = [ {"id": dataset['resource']['id'], 
   "page_views" : dataset['resource']['page_views']['page_views_total'],
   "downloads": dataset['resource']['download_count'],
   "columns" : len(dataset['resource']['columns_name']),
    "name" : dataset['resource']['name'],
    "description" : dataset['resource']['description'],
   'agency': get_agency(dataset),
    "rows": counts[index]
    
  }
 for index,dataset in enumerate(datasets)]

In [64]:
result = (pd.merge(pd.DataFrame(data), 
                  potential_join_datasets, 
                  left_on='id', 
                  right_index=True, 
                  how='left')
           .fillna(0))
result = result.assign(agency=result.agency.astype('category'))

In [65]:
qualitative_colors = sns.color_palette("husl", 30)
qualitative_colors

[(0.9677975592919913, 0.44127456009157356, 0.5358103155058701),
 (0.9694979754285514, 0.4537284387051496, 0.40783466214891867),
 (0.9542900080364486, 0.4779505175890752, 0.1963917622115608),
 (0.8616090647292522, 0.536495730113334, 0.19548899031476086),
 (0.7930046244373694, 0.5708100522446624, 0.19489531984381053),
 (0.7350228985632719, 0.5952719904750953, 0.1944419133847522),
 (0.6804189127793346, 0.6151497514677574, 0.19405452111445337),
 (0.6234809659087222, 0.6330654918766613, 0.19369052933737949),
 (0.5571803870240879, 0.6507747739933957, 0.19331667588333168),
 (0.46810256823426116, 0.6699492535792404, 0.19289587399044988),
 (0.3126890019504329, 0.6928754610296064, 0.1923704830330379),
 (0.19645998900718342, 0.6974309119838096, 0.3614680088321782),
 (0.20125317221201128, 0.6907920815379025, 0.47966761189275336),
 (0.2047934223695329, 0.6857201900461317, 0.5491539606228022),
 (0.20774052347943134, 0.6813852191365138, 0.6002900398972028),
 (0.21044753832183283, 0.6773105080456748, 

In [66]:
qualitative_colors = sns.color_palette("husl", 30)
color_list = dict(zip(
list(result.groupby('agency').count().sort_values(by='page_views', ascending=False).head(30).index),qualitative_colors,))

In [67]:
result = result.assign(r= result.agency.apply(lambda x: color_list[x][0] if x in color_list else 1.0 ),
             g= result.agency.apply(lambda x: color_list[x][1] if x in color_list else 1.0 ),
             b =result.agency.apply(lambda x: color_list[x][2] if x in color_list else 1.0))

In [68]:
result.to_csv('static/dataset_stats.csv',index=False)

In [69]:
with open('color_key.json', 'w') as f:
    json.dump(color_list,f)

In [90]:
get_agency(datasets[10])

'New York City Housing Authority (NYCHA)'

In [74]:
datasets[10]['classification']['domain_metadata']

[{'key': 'Data-Collection_Data-Collection',
  'value': 'Services available to NYCHA Residents - Local Law 163'},
 {'key': 'Update_Automation', 'value': 'No'},
 {'key': 'Update_Date-Made-Public', 'value': '11/29/2019'},
 {'key': 'Update_Update-Frequency', 'value': 'Annually'},
 {'key': 'Dataset-Information_Agency',
  'value': 'New York City Housing Authority (NYCHA)'}]

In [133]:
pos_lookup = dict(zip(list(result.id), list(result.index)))

In [174]:
joins.assign(o = joins.o.apply(lambda x: pos_lookup[x]),
             d = joins.d.apply(lambda x: pos_lookup[x])).rename(columns={'o':'source', 'd': 'target', 'col':'distance'}).to_csv('static/links.csv', index=False)

In [141]:
joins.head()

,o,d,col
0,fuzi-xuv4,dggd-3jfu,1
1,fuzi-xuv4,p5yk-zjsc,6
2,fuzi-xuv4,v2hd-uwxx,3
3,fuzi-xuv4,i6ds-scnc,1
4,fuzi-xuv4,cirn-n726,8
